In [23]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from response.feature import Feature

In [3]:
F_path = "../X_y_data/response2/"
F_name = "Classify_F2.pickle"
featureM = DataManager(F_path)
F = featureM.load_data(F_name)

success load : ../X_y_data/response2/Classify_F2.pickle


In [4]:
model_path = "../models/response2/"
model_name = "Classify_M2.pickle"
modelM = DataManager(model_path)
clf = modelM.load_data(model_name)

success load : ../models/response2/Classify_M2.pickle


In [24]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [25]:
# 質問系かどうかチェックする
usr_sys = []
y = []
errors = ["Ignore question", "Ignore offer", "Ignore proposal", "Ignore greeting"]
for conv in convs:
    for i, ut in enumerate( conv ) :
        if ut.is_system() :
            if ut.is_exist_error() and not ut.is_utt_level_error() :
                usr_sys.append( [conv[i-1].utt, ut.utt] )
                if ut.is_error_included(errors):
                    y.append(1)
                else:
                    y.append(0)


In [13]:
from tqdm import tqdm
print(F.feature_num)
usr = []
for i, u_s in enumerate( tqdm(usr_sys) ):
    x = F.featurization(u_s[0])
    usr.append(x)
usr_ = np.array(usr)

  0%|          | 0/976 [00:00<?, ?it/s]

56198


100%|██████████| 976/976 [02:59<00:00,  5.45it/s]


In [14]:
question_pred = clf.predict(usr_)
# np.count_nonzero(question_pred) : 643

In [26]:
from sentence_transformers import SentenceTransformer
bert_path = "../../corpus/pretrained/sbert_ignore"
sbert = SentenceTransformer(bert_path)

In [27]:
smodel_path = "../models/response2/"
smodel_name = "Classify_M_sbert.pickle"
smodelM = DataManager(smodel_path)

sclf = smodelM.load_data(smodel_name)

success load : ../models/response2/Classify_M_sbert.pickle


In [28]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)

def text2feature(text):
    vector = text2vec(text)
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])

In [29]:

question_X = []
for u_s, qp in zip(usr_sys, question_pred):
    # if qp==1:
    #     question_X.append(u_s)
    question_X.append(u_s)

In [30]:
X_sbert = [ text2feature(x) for x in question_X]
sbeert_pred = sclf.predict(X_sbert)

In [31]:
count = 0
y_pred = []
for p in question_pred:
    if p==1:
        if sbeert_pred[count] <2:
            y_pred.append(0)
        else:
            y_pred.append(1)
        count+=1
    else:
        y_pred.append(0)

In [32]:
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[608  14]
 [235 119]]
accuracy =  0.7448770491803278
precision =  0.8947368421052632
recall =  0.3361581920903955
f1 score =  0.4887063655030801


## 新バージョン 学習方法変更
- 前向きの検出 and S-BERT を用いた検出

        confusion matrix = 
        [[878 154]
        [ 82 272]]
        accuracy =  0.8297258297258298
        precision =  0.6384976525821596
        recall =  0.768361581920904
        f1 score =  0.6974358974358975

- 発話レベルエラーを除く(誤情報が邪魔？)

        confusion matrix = 
        [[585  37]
        [ 80 274]]
        accuracy =  0.8801229508196722
        precision =  0.8810289389067524
        recall =  0.7740112994350282
        f1 score =  0.8240601503759398

- S-BERT のみ

        confusion matrix = 
        [[608  14]
        [235 119]]
        accuracy =  0.7448770491803278
        precision =  0.8947368421052632
        recall =  0.3361581920903955
        f1 score =  0.4887063655030801


## 旧バージョン

- 前向きの検出 and S-BERT を用いた検出

        confusion matrix = 
        [[1654  192]
        [ 163  191]]
        accuracy =  0.8386363636363636
        precision =  0.49869451697127937
        recall =  0.53954802259887
        f1 score =  0.5183175033921302

- 前向きの検出 and S-BERT を用いた検出 and 特徴量変更後

        confusion matrix = 
        [[1647  199]
        [ 145  209]]
        accuracy =  0.8436363636363636
        precision =  0.5122549019607843
        recall =  0.5903954802259888
        f1 score =  0.5485564304461943

- S-BERT のみ

        confusion matrix = 
        [[1542  304]
        [  79  275]]
        accuracy =  0.8259090909090909
        precision =  0.4749568221070812
        recall =  0.7768361581920904
        f1 score =  0.5894962486602358

- SBERT のみ and 特徴量変更後

        confusion matrix = 
        [[1521  325]
        [  61  293]]
        accuracy =  0.8245454545454546
        precision =  0.4741100323624595
        recall =  0.827683615819209
        f1 score =  0.602880658436214